In [1]:
# import libraries
import numpy as np
import cv2 as cv

In [2]:
# read in single test image
img = cv.imread("/home/leo/Documents/UNI/ICDS/Groupwork/Group1/data/test/normal/IM-0115-0001.jpeg", cv.IMREAD_GRAYSCALE)
print(img.shape)

(1858, 2090)


In [7]:
class conv_layer:
    def __init__(self, in_dim, conv_size=(3,3), conv_num=4):
        self.conv_size = conv_size
        self.conv_kernels = []
        for i in range(0, conv_num):
            self.conv_kernels[i] = np.empty(conv_size)
        self.in_dim = in_dim
        self.out_dim = (in_dim[0] - (conv_size[0] - 1), in_dim[1] - (conv_size[1] - 1, conv_num))
        
    def forward(self, img):
        out_img = np.empty(self.out_dim)
        for i in range(0, len(self.conv_kernels)):
            for ii in range(0, self.out_dim[0]):
                for iii in range(0, self.out_dim[1]):
                    out_img[ii,iii, len(self.conv_kernels)] = np.sum(img[ii:ii+self.conv_size[0], iii:iii+self.conv_size[1]] * self.conv[i])
        return out_img

In [4]:
class pooling_layer:
    def __init__(self, in_dim, pooling_size=(3,3)):
        self.pooling_size = pooling_size
        self.in_dim = in_dim
        self.out_dim = (in_dim[0] - (pooling_size[0] - 1), in_dim[1] - (pooling_size[1] - 1))
        
    def forward(self, img):
        out_img = np.empty(self.out_dim)
        for i in range(0, self.out_dim[0]):
            for ii in range(0, self.out_dim[1]):
                out_img[i,ii] = np.max(img[i:i+self.pooling_size[0], ii:ii+self.pooling_size[1]])
        return out_img

In [ ]:
class linear_layer:
    def __init__(self, in_dim):
        pass

In [8]:
# test conv layer
a = np.zeros((4,4))
index = 0
for ii in range(0,a.shape[0]):
    for iii in range(0,a.shape[1]):
        a[ii, iii] = index
        index += 1

print("input before pool")
print(a)
c = pooling_layer((4,4))
d = c.forward(a)
print("output pool")
print(d)

input before pool
[[ 0.  1.  2.  3.]
 [ 4.  5.  6.  7.]
 [ 8.  9. 10. 11.]
 [12. 13. 14. 15.]]
output pool
[[10. 11.]
 [14. 15.]]


In [9]:
# test conv layer
a = np.zeros((4,4))
index = 0
for ii in range(0,a.shape[0]):
    for ii in range(0,a.shape[1]):
        a[ii, ii] = index
        index += 1

print("input before conv")
print(a)
c = conv_layer((4,4))
d = c.forward(a)
print("output conv")
print(d)

input before conv
[[12.  0.  0.  0.]
 [ 0. 13.  0.  0.]
 [ 0.  0. 14.  0.]
 [ 0.  0.  0. 15.]]


IndexError: list assignment index out of range

In [ ]:
# test them in succesion

